In [ ]:

# Make Predictions with k-nearest neighbors on the Iris Flowers Dataset
from csv import reader
from math import sqrt
 
# Load a CSV file
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	return dataset
 
'''# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())
 
# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
		print('[%s] => %d' % (value, i))
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup
 
# Find the min and max values for each column
def dataset_minmax(dataset):
	minmax = list()
	for i in range(len(dataset[0])):
		col_values = [row[i] for row in dataset]
		value_min = min(col_values)
		value_max = max(col_values)
		minmax.append([value_min, value_max])
	return minmax
 
# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
	for row in dataset:
		for i in range(len(row)):
			row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])
 '''
# Calculate the Euclidean distance between two vectors
def euclidean_distance(row1, row2):
	distance = 0.0
	for i in range(len(row1)-1):
		distance += (row1[i] - row2[i])**2
	return sqrt(distance)
 
# Locate the most similar neighbors
def get_neighbors(train, test_row, num_neighbors):
	distances = list()
	for train_row in train:
		dist = euclidean_distance(test_row, train_row)
		distances.append((train_row, dist))
	distances.sort(key=lambda tup: tup[1])
	neighbors = list()
	for i in range(num_neighbors):
		neighbors.append(distances[i][0])
	return neighbors
 
# Make a prediction with neighbors
def predict_classification(train, test_row, num_neighbors):
	neighbors = get_neighbors(train, test_row, num_neighbors)
	output_values = [row[-1] for row in neighbors]
	prediction = max(set(output_values), key=output_values.count)
	return prediction
 
# Make a prediction with KNN on Iris Dataset
'''filename = 'iris.csv'
dataset = load_csv(filename)
for i in range(len(dataset[0])-1):
	str_column_to_float(dataset, i)
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)'''
# define model parameter
num_neighbors = 5
# define a new record
row = [5.7,2.9,4.2,1.3]
# predict the label
label = predict_classification(dataset, row, num_neighbors)
print('Data=%s, Predicted: %s' % (row, label))

In [1]:
!pip install ipython-autotime
%load_ext autotime

time: 1.12 ms (started: 2023-03-28 19:21:28 +05:30)


In [ ]:
!pip install nufhe[pycuda]
!pip install pycuda

     |████████████████████████████████| 86 kB 2.9 MB/s 
     |████████████████████████████████| 189 kB 31.7 MB/s 
     |████████████████████████████████| 1.7 MB 41.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 75 kB 4.1 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
  Created wheel for pycuda: filename=pycuda-2021.1-cp37-cp37m-linux_x86_64.whl size=627464 sha256=8228c4589d9ec66e1a5efcb6a05ec35f51d9d482bc51f8c73d47e029d049ce2a
  Stored in directory: /root/.cache/pip/wheels/c4/ef/49/dc6a5feb8d980b37c83d465ecab24949a6aa19458522a9e001
  Created wheel for pytools: filename=pytools-2021.2.8-py2.py3-none-any.whl size=60725 sha256=ad9c61a35e94e67ca8bf4f4fc26177934f02812c2f279cf8ab37367c92457db5
  Stored in directory: /root/.cache/pip/wheels/bd/2d/ef/0127a17bafa44971f11d05d0e38d7947144cf9e33313bf12a7
  Created wheel for reikna: filename=reikna-0.7.5-py3-

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 23.2 s (started: 2021-10-24 16:05:02 +00:00)


In [35]:
import pandas as pd


0    51
1    25
2    30
3    11
4     1
Name: 0, dtype: int64
0    47
1    32
2    13
3     2
4    -1
Name: 1, dtype: int64
0    57
1    28
2    41
3    13
4     1
Name: 2, dtype: int64
0    46
1    31
2    15
3     2
4    -1
Name: 3, dtype: int64
0    50
1    36
2    14
3     2
4    -1
Name: 4, dtype: int64
0    56
1    27
2    42
3    13
4     1
Name: 5, dtype: int64
0    57
1    30
2    42
3    12
4     1
Name: 6, dtype: int64
0    50
1    34
2    15
3     2
4    -1
Name: 7, dtype: int64
0    44
1    29
2    14
3     2
4    -1
Name: 8, dtype: int64
0    56
1    30
2    41
3    13
4     1
Name: 9, dtype: int64


In [1]:
import time
import random
#import pycuda.autoinit
import nufhe
import numpy as np
import numpy as np
from datetime import datetime
def subtractBits(r, a, b, carry):
    # Xor(t1[0], a, carry[0])
    t1 = vm.gate_xor(a, b)      #axorb
    # Xor(t2[0], b, carry[0])
    # Xor(r[0], a, t2[0])
    r[0] = vm.gate_xor(t1, carry) #axorbxorc
    # And(t1[0], t1[0], t2[0])
    acomp=vm.gate_not(a)   #a'
    abcomp=vm.gate_not(t1)  #axorb'
    t2 = vm.gate_and(acomp, b)
    t3 = vm.gate_and(abcomp, carry)
    #t4=vm.gate_and(t2,t3)
    #t5= vm.gate_or(t2,t3)
    # Xor(r[1], carry[0], t1[0])
    r[1] = vm.gate_or(t2, t3)

    return r
def subtractNumbers(ciX, ciY,minus_one):
# fixed iteration since
    one_ciY=vm.gate_not(ciY)
    comp_ciY=add(one_ciY,one)
    #result=ctx.decrypt(secret_key,comp_ciY)
    #val=listToString(result)
    #print("2scomplement of one ",two2dec(val[::-1]) )
    for i in range(size):
        a = vm.gate_and(ciX, comp_ciY)
        b = vm.gate_xor(ciX, comp_ciY)
        aShiftTemp = a
        # using roll as a shift bit
        aShiftTemp.roll(1, axis=0)
        #aShiftTemp=np.roll(aShiftTemp,-1, axis=0)
        ciX = aShiftTemp
        comp_ciY = b
    #print("after result is",ctx.decrypt(secret_key,b))    
    for i in range(size):
        c_s[i]=ciY[size-1]
    not_minus=vm.gate_not(minus_one)
    comp_minus=add(not_minus,one)
    comp_b=add(b,comp_minus)
    #print("comp_b result is",ctx.decrypt(secret_key,comp_b))
    #result=ctx.decrypt(secret_key,comp_b)
    #val=listToString(result)
    #print("comp is",two2dec(val[::-1]) )
    #print("control bit",ctx.decrypt(secret_key,c_s))
    b=mux(c_s,b,comp_b)
    #result=ctx.decrypt(secret_key,b)
    #val=listToString(result)
    #print("after add ",two2dec(val[::-1]) )
    return b







'''def subtractNumbers(ctA, ctB, nBits):
    ctRes = [[vm.empty_ciphertext((1,))] for i in range(0,nBits)]
    # carry = vm.empty_ciphertext((1,))
    bitResult = [[vm.empty_ciphertext((1,))] for i in range(2)]
    ctRes[0] = vm.gate_xor(ctA[0], ctB[0])
    # Xor(ctRes[0], ctA[0], ctB[0])
    t1=vm.gate_not(ctA[0])
    carry = vm.gate_and(t1, ctB[0])
    # And(carry[0], ctA[0], ctB[0])
    for i in range(1,nBits ):
        bitResult = subtractBits(bitResult, ctA[i], ctB[i], carry)
        # Copy(ctRes[i], bitResult[0]);
        ctRes[i] = nufhe.LweSampleArray.copy(bitResult[0])

        # Copy(carry[0], bitResult[1])
        carry = nufhe.LweSampleArray.copy(bitResult[1])

    return ctRes'''

def predict_classification(train, test_row, num_neighbors):
	neighbors = get_neighbors(train, test_row, num_neighbors)
	output_values = [row[-1] for row in neighbors]
	prediction = max(set(output_values), key=output_values.count)
	return prediction
#def bubblesort(distances)

def get_neighbors(train, test_row, num_neighbors):
	distances = list()
	for train_row in train:
		dist = euclidean_distance(test_row, train_row,nbits)
		distances.append((train_row, dist))
	distances.sort(key=lambda tup: tup[1])
	neighbors = list()
	for i in range(num_neighbors):
		neighbors.append(distances[i][0])
	return neighbors




def euclidean_distance(row1, row2,minus_one,nBits):
    ones= ctx.encrypt(secret_key, [[False] for _ in range(nBits)])
    distance=  ctx.encrypt(secret_key, [[False] for _ in range(nBits)])
    signbit=  ctx.encrypt(secret_key, [[False] for _ in range(nBits)])
    c_zero= ctx.encrypt(secret_key, [[False] for _ in range(nBits)])
    #distance = 0.0
    '''for i in range(nBits):
        temp=vm.gate_copy(row2[i])
        ones[i]=vm.gate_not(temp)
    neg2=make_neg(ones,nBits)'''
    
    
    result=subtractNumbers(row1,row2,minus_one)
    for i in range(nBits):
        signbit[i]=result[nBits-1]
    
        
    ones=vm.gate_not(result)
    neg2=add(ones,one)      
    print("signbit type",type(signbit))
    print("neg2 is",type(neg2))
    distance=mux(signbit,neg2,result)
    return distance



def make_neg(n,minus_one,nBits):
    list1=[int(i) for i in range(0,size) ]
    listp=[]
    #one=vm.empty_ciphertext((1,))
    #zero=[vm.empty_ciphertext((1,))]
    #one= [ctx.encrypt(secret_key, [True])]
    #zero= [ctx.encrypt(secret_key, [False])]
    #t2= [ctx.encrypt(secret_key, [False])]
    #one=  [ctx.encrypt(secret_key, [False]) for _ in range(nbits)]
    #zero=  [ctx.encrypt(secret_key, [False) for _ in range(nbits)]
    #print("type of one is",type(one))
    #print("typeof zero is",type(zero))
    #print("tupe of one is",one)
    #[vm.empty_ciphertext((1,))]
    '''for i in range(0,len(n)):
        #temp=vm.gate_copy(n[i])
        temp=vm.gate_mux(n[i],zero[0],one[0])
        n[i]=temp[:]'''
            
    
    #print(n)
    one= ctx.encrypt(secret_key, [True])
    onep=  ctx.encrypt(secret_key, [[False] for _ in range(nBits)])
    onep[0]=vm.gate_copy(one) 
    testone= ctx.encrypt(secret_key, [[False] for _ in range(nBits)])
    testone=onep
    '''temp1=n[:]
    temp1.reverse()
    result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(nBits)]
    pa=listToString(result_bits)
    print("n before",twos_comp_val(int(pa,2),len(pa)))'''
    n=addNumbers(n,testone,minus_one,nBits)
    '''temp1=n[:]
    temp1.reverse()
    result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(nBits)]
    pa=listToString(result_bits)
    print("n after",twos_comp_val(int(pa,2),len(pa)))'''
    return n


def fixSizeBoolList(decimal,size):
    x = [int(x) for x in bin(decimal)[2:]]
    x = list(map(bool, x))
    x = [False]*(size - len(x)) + x
    pow2 = []
    for i in range(size):
        pow2.append([x[i]])
    pow2.reverse()
    return pow2
def boolListToInt(bitlists):
    out = 0
    for bit in bitlists:
        out = (out << 1) | bit
    return out

def addBits(r, a, b, carry):
    # Xor(t1[0], a, carry[0])
    t1 = vm.gate_xor(a, b)
    # Xor(t2[0], b, carry[0])
    # Xor(r[0], a, t2[0])
    r[0] = vm.gate_xor(t1, carry)
    # And(t1[0], t1[0], t2[0])
    t2 = vm.gate_and(a, carry)
    t3 = vm.gate_and(b, carry)
    t4=vm.gate_and(a,b)
    t5= vm.gate_or(t2,t3)
    # Xor(r[1], carry[0], t1[0])
    r[1] = vm.gate_or(t5, t4)

    return r
def addNumbers(ctA, ctB,minus_one, nBits):
    ctRes = [[vm.empty_ciphertext((1,))] for i in range(nBits)]
    # carry = vm.empty_ciphertext((1,))
    bitResult = [[vm.empty_ciphertext((1,))] for i in range(2)]
    ctRes[0] = vm.gate_xor(ctA[0], ctB[0])
    # Xor(ctRes[0], ctA[0], ctB[0])
    carry = vm.gate_and(ctA[0], ctB[0])
    # And(carry[0], ctA[0], ctB[0])
    for i in range(1,nBits ):
        if i>0:
            bitResult = addBits(bitResult, ctA[i], ctB[i], carry)
            # Copy(ctRes[i], bitResult[0]);
            ctRes[i] = nufhe.LweSampleArray.copy(bitResult[0])
            # Copy(carry[0], bitResult[1])
            carry = nufhe.LweSampleArray.copy(bitResult[1])
        else:
            #bitResult = twotoone(bitResult, ctA[i], ctB[i])
            bitResult = threetotwo(bitResult, ctA[i], ctB[i],carry)
            #Copy(ctRes[i], bitResult[0]);
            ctRes[i] = nufhe.LweSampleArray.copy(bitResult[0])
            carry = nufhe.LweSampleArray.copy(bitResult[1])
        
    return ctRes
def Convert_list(string):
    list1=[]
    list1[:0]=string
    #print(list1)
    list1=[int(i)for i in list1 ]
    listb=[]
    for i in list1:
        if i==0:
            listb.append([False])
        else:
            listb.append([True])    
    
    #print(listb)
    return listb

def twos_complement(n,nbits):
    a=f"{n & ((1 << nbits) - 1):0{nbits}b}" 
    #print(type(a))
    a=Convert_list(a)
    a.reverse()
    return a
def listToString(s): 
    # initialize an empty string
    list1=[int(i)for i in s ]
    listp=[]
    for i in list1:
        if i==False:
            listp.append('0')
        else:
            listp.append('1')    
    
    #print(listp)
    str1 = "" 
    # traverse in the string  
    s=['delim'.join([str(elem) for elem in sublist]) for sublist in listp]
    #print(s)
    for ele in s: 
        str1 += ele  
    # return string 
    #print(str1)
    return str1
def twos_comp_val(val,bits):
    """compute the 2's complement of int value val"""
    #val=listToString(val)
    

    if (val & (1 << bits - 1)) != 0: # if sign bit is set e.g., 8bit: 128-255
        val = val - (1 << bits)        # compute negative value
    return val

def compare_bit(  a, b,  lsb_carry,  tmp): 
    result= ctx.encrypt(secret_key, [False])
    tmp=vm.gate_xnor(a, b)
    result=vm.gate_mux(tmp,lsb_carry, a)
    return result


def minimum(  a,  b,  nb_bits): 
    tmps1= ctx.encrypt(secret_key, [False])
    tmps2= ctx.encrypt(secret_key, [True])
    #initialize the carry to 0
    #run the elementary comparator gate n times
    for i in range(nb_bits):
        tmps1= compare_bit(a[i],b[i],tmps1,tmps2)
    #tmps[0] is the result of the comparaison: 0 if a is larger, 1 if b is larger
    #select the max and copy it to the result
    return tmps1

    
        
def predict(ctA,secret_key, output_bits):
    zero = [ctx.encrypt(secret_key, [False]) for _ in range(output_bits)]
    onen=  [ctx.encrypt(secret_key, [True]) for _ in range(output_bits)]        
    onep=  [ctx.encrypt(secret_key, [False]) for _ in range(output_bits)]
    one= ctx.encrypt(secret_key, [True])
    temp= ctx.encrypt(secret_key, [True])
    temp=ctA[output_bits-1]
    comp_res= ctx.encrypt(secret_key, [True])
    onep[0]=one
    ctRes = [ctx.encrypt(secret_key, [False]) for _ in range(output_bits)]
    # Copy(ctRes[i], bitResult[0]);
    #comp_res= minimum(ctA,zero,output_bits)
    #temp=
    #comp_res=
    
    
    
    for i in range(output_bits):
        ctRes[i] = vm.gate_mux(temp,onen[i],onep[i])  
    # Copy(carry[0], bitResult[1])
    return ctRes


def dist_calc(line,line1,nbits):
    #print(line)
    print(type(list))
    
    return wx,b_l
def add(ciX, ciY):
# fixed iteration since
    
    for i in range(size):
        a = vm.gate_and(ciX, ciY)
        b = vm.gate_xor(ciX, ciY)
        aShiftTemp = a
        # using roll as a shift bit
        aShiftTemp.roll(1, axis=0)
        #aShiftTemp=np.roll(aShiftTemp,-1, axis=0)
        ciX = aShiftTemp
        ciY = b
    #result=ctx.decrypt(secret_key,b)
    #print("result is",boolListToInt(result))
    return b
def mux(c_s,true,false) :
    temp1=vm.gate_and(c_s,true)
    #print("temp1:",ctx.decrypt(secret_key, temp1))
    temp2=vm.gate_and(vm.gate_not(c_s),false)
    #print("temp2:",ctx.decrypt(secret_key, temp2))
    mux_result=vm.gate_or(temp1,temp2)
    return mux_result
def two2dec(s):
    if s[0] == '1':
        return -1 * (int(''.join('1' if x == '0' else '0' for x in s), 2) + 1)
    else:
        return int(s, 2)











if __name__ == '__main__':    
    ctx = nufhe.Context()
    secret_key, cloud_key = ctx.make_key_pair()
    vm = ctx.make_virtual_machine(cloud_key)
    size=16
    nbitis=16
    onep=  ctx.encrypt(secret_key, [[False] for _ in range(size)])
    onen=  ctx.encrypt(secret_key, [[True] for _ in range(size)])
    temp_dist=  [ctx.encrypt(secret_key, [False]) for _ in range(size)]
    temp_label=  [ctx.encrypt(secret_key, [False]) for _ in range(size)]
    one= ctx.encrypt(secret_key, [True])
    zero=ctx.encrypt(secret_key, [False])
    onep[0]=vm.gate_copy(one)
    with open('/home/pradeep/Desktop/train.txt') as f:
        lines = []
        for line in f:
            lines.append([int(v) for v in line.split()])
    with open('/home/pradeep/Desktop/test.txt') as f:
        lines1 = []
        for line in f:
            lines1.append([int(v) for v in line.split()])
    bits = [[False] for i in range(size - 2)]
    zeros = [[False] for i in range(size)]
    train_size= int(input("Please enter train a string:\n"))
    test_size=int(input("Please enter test data a string:\n"))
    neigh=int(input("Please enter no of neighbours:\n"))
    minus_one=ctx.encrypt(secret_key, twos_complement(-1,size))
    s=[ [False] for i in range(size)]
    c_s=ctx.encrypt(secret_key, s)
    '''b_x0=[]
    b_x1=[]
    b_x2=[]
    b_x3=[]
    for i in range(test_size):
        temp=int(lines[i][0])
        #print(type(temp))
        b_x0.append(fixSizeBoolList(temp,size))
        temp=int(lines[i][1])
        b_x1.append(fixSizeBoolList(temp,size))
        temp=int(lines[i][2])
        b_x2.append(fixSizeBoolList(temp,size))
        temp=int(lines[i][3])
        b_x3.append(fixSizeBoolList(temp,size))'''
    #print(b_x0,b_x1,b_x2,b_x3)
    #-3.5133618, -4.2616279 , 10.59763759,  6.17913037, -0.1758181
    #w_p=[-15,-18 ,-78 ,138 , 62] # scratch x/1+|x|
    w_p=[-1,-3,4,2,0] # sk learn x/1+|x|
    #w_p=[4,-8 ,22 ,9 , -1] sigmoid
    #w_p=[-3, -4 , 10,  6, -1] svm
    
    #print(type(x[0]))
    #print(x)
    #b_y = fixSizeBoolList(deci_y,size)
    #print(type(y[0]))
    #print(y)
    #x.reverse()
    #print(x)
    #y.reverse()
    #print(y)
    featuresize=5
    '''w=[]
    w_b=[]
    for i in range(featuresize-1):
        w.append([[vm.empty_ciphertext((1,))] for i in range(size)])
        w_b.append(twos_complement(w_p[i],size))
        for j in range(size):
            w[i][j] = ctx.encrypt(secret_key, w_b[i][j])
    bias=[[vm.empty_ciphertext((1,))] for i in range(size)]
    bias_b=twos_complement(w_p[4],size)'''
    '''eten=[[vm.empty_ciphertext((1,))] for i in range(size)]
    eten_b=twos_complement(10,size)
    for i in range(size):
        bias[i]=ctx.encrypt(secret_key, bias_b[i])
        eten[i]=ctx.encrypt(secret_key, eten_b[i])
    '''
    #train_data = pd.read_csv('/home/pradeep/Desktop/train.txt',sep=" ",header = None)
    #test_data=pd.read_csv('/home/pradeep/Desktop/test.txt',sep=" ",header = None)
    #dist=[]
    #for i in range(train_size):
    #    dist.append([[ctx.encrypt(secret_key, [False]) for i in range(16)] for j in range(2)]) 
    dist=[[ctx.encrypt(secret_key,[[False] for _ in range(size)]) for j in range(2)]for k in range(train_size)]
    plain_predict=[]
    start_time = datetime.now()
    #dist=[[vm.empty_ciphertext((1,))] for i in range(train_size) for j in range(size)]
    for j in range(test_size):
        print("j values is",j)
        print("################################")
        for i in range(train_size):
            print("i value is ",i)
            print(lines[i][0],lines[i][1],lines[i][2],lines[i][3],lines[i][4])
            print(lines1[j][0],lines1[j][1],lines1[j][2],lines1[j][3])
            temp=int(lines[i][0])
            b_x0=twos_complement(temp,size)
            temp=int(lines[i][1])
            b_x1=twos_complement(temp,size)
            temp=int(lines[i][2])
            b_x2=twos_complement(temp,size)
            temp=int(lines[i][3])
            b_x3=twos_complement(temp,size)
            temp=int(lines[i][4])
            b_l=twos_complement(temp,size)
            temp1=int(lines1[j][0])
            b_y0=twos_complement(temp1,size)
            temp1=int(lines1[j][1])
            b_y1=twos_complement(temp1,size)
            temp1=int(lines1[j][2])
            b_y2=twos_complement(temp1,size)
            temp1=int(lines1[j][3])
            b_y3=twos_complement(temp1,size)
            
            
            ciphertext1 = ctx.encrypt(secret_key, b_x0)
            ciphertext2 = ctx.encrypt(secret_key, b_x1)
            ciphertext3 = ctx.encrypt(secret_key, b_x2)
            ciphertext4 = ctx.encrypt(secret_key, b_x3)
            ciphertextl = ctx.encrypt(secret_key, b_l)
            ciphertext1a = ctx.encrypt(secret_key, b_y0)
            ciphertext2a = ctx.encrypt(secret_key, b_y1)
            ciphertext3a = ctx.encrypt(secret_key, b_y2)
            ciphertext4a = ctx.encrypt(secret_key, b_y3)
            #ciphertext1 = ctx.encrypt(secret_key, x)
            #ciphertext2 = ctx.encrypt(secret_key, y) 
            start_time = time.time()
            #ciphertext1.reverse()
            #ciphertext2.reverse()
            #result = addNumbers(ciphertext1, ciphertext2, size)
            #partial_predict=[]

            #print(type(ciphertext1[0][0]))
            temp1 = [[vm.empty_ciphertext((1,))] for i in range(size)]
            temp2 = [[vm.empty_ciphertext((1,))] for i in range(size)]
            #wx=[[vm.empty_ciphertext((1,))] for i in range(size)]
            #predict[i]=[[vm.empty_ciphertext((1,))] for i in range(size*2)]
            #temp1=[]
            #temp2=[]
            presult_mul1 = euclidean_distance(ciphertext1,ciphertext1a,minus_one, size)
            #print("ciphertext one",type(ciphertext1))
            #print("ciphertext one inside",type(ciphertext1[0]))
            '''temp1=presult_mul1[:]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)
            print("mul1",twos_comp_val(int(pa,2),len(pa)))'''
            presult_mul2 = euclidean_distance(ciphertext2,ciphertext2a ,minus_one,size)
            '''temp1=presult_mul2[:]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)
            print("mul2",twos_comp_val(int(pa,2),len(pa)))'''
            presult_mul3 = euclidean_distance(ciphertext3,ciphertext3a,minus_one,  size )
            '''temp1=presult_mul3[:]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)
            print("mul3",twos_comp_val(int(pa,2),len(pa)))'''
            presult_mul4 = euclidean_distance(ciphertext4, ciphertext4a,minus_one,  size)
            '''temp1=presult_mul4[:]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)
            print("mul4",twos_comp_val(int(pa,2),len(pa)))'''
            presult_add1 = addNumbers(presult_mul1, presult_mul2,minus_one,  size)
            '''temp1=presult_add1[:]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)
            print("firstadd",pa)
            print("add1",twos_comp_val(int(pa,2),len(pa)))'''
            presult_add2 = addNumbers(presult_mul3, presult_mul4,minus_one,  size)
            '''temp1=presult_add2[:]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)
            #print("secondadd",pa)
            print("add2",twos_comp_val(int(pa,2),len(pa)))'''
            dist[i]= addNumbers(presult_add1, presult_add2,minus_one,  size)
            '''temp1=dist[i][0][:]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)
            print("dist is",twos_comp_val(int(pa,2),len(pa)))'''
            #print(type(temp))
            #x = twos_complement(deci_x,size)
            dist[i][1]=ciphertextl[:]
            
            
        for l in range( train_size):    
            # Last i elements are already in place
            for m in range(train_size-l-1):
                #if (arr[j] > arr[j+1]):
                #swap(&arr[j], &arr[j+1]);
                temp=subtractNumbers(dist[m][0],dist[m+1][0], size)
                ''''temp1=temp[:]
                temp1.reverse()
                result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
                pa=listToString(result_bits)
                print("temp",twos_comp_val(int(pa,2),len(pa)))
                '''
                signbit= ctx.encrypt(secret_key, [False])
                signbitnot= ctx.encrypt(secret_key, [False])
                signbit=temp[size-1]
                signplain=[]
                signplain.append([ctx.decrypt(secret_key,signbit)])
                print("signbit is",signplain)
                signbitnot=vm.gate_not(signbit)
                tempb1=[ctx.encrypt(secret_key, [False]) for _ in range(size)]
                tempb2=[ctx.encrypt(secret_key, [False]) for _ in range(size)]
                tempb3=[ctx.encrypt(secret_key, [False]) for _ in range(size)]
                tempb4=[ctx.encrypt(secret_key, [False]) for _ in range(size)]
                
                for _ in range(size):
                    tempb1[_]=dist[m][0][_]
                    tempb2[_]=dist[m+1][0][_]
                    tempb3[_]=dist[m][1][_]
                    tempb4[_]=dist[m+1][1][_]
                for k in range(size):
                    dist[m][0][k]=vm.gate_mux(signbit,dist[m][0][k],dist[m+1][0][k])
                    dist[m+1][0][k]=vm.gate_mux(signbit,tempb2[k],tempb1[k])
                    dist[m][1][k]=vm.gate_mux(signbit,dist[m][1][k],dist[m+1][1][k])
                    dist[m+1][1][k]=vm.gate_mux(signbit,tempb4[k],tempb3[k]) 
                temp1=dist[m][0][:]
                temp1.reverse()
                result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
                pa=listToString(result_bits)
                print("m0",twos_comp_val(int(pa,2),len(pa)))
                temp1=dist[m+1][0][:]
                temp1.reverse()
                result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
                pa=listToString(result_bits)
                print("m1",twos_comp_val(int(pa,2),len(pa)))
                
                '''
                temp1=dist[m][0]
                temp1.reverse()
                result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
                pa=listToString(result_bits)
                print("wx is",pa)
                print(" m valus is",m,"wx",twos_comp_val(int(pa,2),len(pa)))'''
        pos=ctx.encrypt(secret_key, [[False] for _ in range(size)])
        neg=ctx.encrypt(secret_key, [[False] for _ in range(size)])
        #predict1=[ctx.encrypt(secret_key, [False]) for _ in range(size)]
        tempp=ctx.encrypt(secret_key, [[False] for _ in range(size)])
        posc=ctx.encrypt(secret_key, [[False] for _ in range(size)])
        negc=ctx.encrypt(secret_key, [[False] for _ in range(size)])
        dbit=ctx.encrypt(secret_key, [[False] for _ in range(size)])
        signbitl=ctx.encrypt(secret_key, [[False] for _ in range(size)])
        signbitc=ctx.encrypt(secret_key, [[False] for _ in range(size)])
        for i in range(neigh):
            for j in range(size):
                signbitl[j]=dist[i][1][size-1]
                signbitc[j]=onep[size-1]
            print("dist type is",type(dist[i][1][size-1]))
            print("onep type is",type(onep))
            print("dbit type is",type(dbit))
            dbit=vm.gate_xnor(signbitl,signbitc)
            posc=pos
            negc=neg
            one_ciY=vm.gate_not(pos)
            pos_comp=add(one_ciY,one)
            one_ciY=vm.gate_not(neg)
            neg_comp=add(one_ciY,one)
            #pos_copy=make_neg(pos,minus_one,size)
            #neg_copy=make_neg(neg,minus_one,size)
            #for j in range(size):
            print("dbit type is",type(dbit))    
            pos=mux(dbit,posc,pos_comp)
            neg=mux(dbit,neg_comp,negc)     
        temp=subtractNumbers(pos,neg,minus_one)
        signbit=ctx.encrypt(secret_key, [[False] for _ in range(size)])
        for i in range(size):
            signbit[i]= vm.gate_copy(temp[size-1])
        
        
        tempp=vm.gate_mux(signbit,onen,onep)
        x=ctx.decrypt(secret_key,tempp)
        val=listToString(x)
        print("predicted ",two2dec(val[::-1]))
        plain_predict.append(twos_comp_val(int(pa,2),len(pa)))
        print("endtime",datetime.now(),"start time",start_time)   
    #print("start time is",start_time)
    #print("end time",time.time())
    print("endtime",datetime.now(),"start time",start_time)
    #result.reverse()
    for j in range(test_size):
        print(" nuFHE multiplication number is actual",lines[j][4],"preicted", plain_predict[j])

Please enter train a string:
5
Please enter test data a string:
1
Please enter no of neighbours:
3
j values is 0
################################
i value is  0
51 25 30 11 1
66 29 46 13
signbit type <class 'nufhe.lwe.LweSampleArray'>
neg2 is <class 'nufhe.lwe.LweSampleArray'>
signbit type <class 'nufhe.lwe.LweSampleArray'>
neg2 is <class 'nufhe.lwe.LweSampleArray'>


KeyboardInterrupt: 

In [4]:
import time
import random
#import pycuda.autoinit
import nufhe
import numpy as np
import numpy as np
from datetime import datetime
ctx = nufhe.Context()
secret_key, cloud_key = ctx.make_key_pair()
vm = ctx.make_virtual_machine(cloud_key)
size=16
dist=[]
for i in range(2):
        dist.append([[ctx.encrypt(secret_key, [False]) for i in range(16)] for j in range(2)])
print(dist[0][1])        

[<nufhe.lwe.LweSampleArray object at 0x7fac845cee20>, <nufhe.lwe.LweSampleArray object at 0x7fac6803f7f0>, <nufhe.lwe.LweSampleArray object at 0x7fac48589dc0>, <nufhe.lwe.LweSampleArray object at 0x7fac48310eb0>, <nufhe.lwe.LweSampleArray object at 0x7fac483fb9a0>, <nufhe.lwe.LweSampleArray object at 0x7fac48537bb0>, <nufhe.lwe.LweSampleArray object at 0x7fac6801fd00>, <nufhe.lwe.LweSampleArray object at 0x7fac48555100>, <nufhe.lwe.LweSampleArray object at 0x7fac484bb280>, <nufhe.lwe.LweSampleArray object at 0x7fac68014130>, <nufhe.lwe.LweSampleArray object at 0x7fac680145b0>, <nufhe.lwe.LweSampleArray object at 0x7fac845fcfd0>, <nufhe.lwe.LweSampleArray object at 0x7fac48417d60>, <nufhe.lwe.LweSampleArray object at 0x7fac48417670>, <nufhe.lwe.LweSampleArray object at 0x7fac48417eb0>, <nufhe.lwe.LweSampleArray object at 0x7fac485ce580>]
